In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON
from SPARQLWrapper import SPARQLWrapper2
import os.path
import re
import time

In [12]:
# Tenemos que realizar la consulta utilizando la función OFFSET, debido a que ·
# la cantidad de 'birds' encontrada es mayor a 10000, con este parámetro nos  ·
# permite iniciar la segunda consulta a partir de la observación número 10000 ·
# para así tener toda la muestra                                              ·
# ·············································································

sparql_1 = SPARQLWrapper2("http://dbpedia.org/sparql")
sparql_1.setQuery("""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT * WHERE {
    ?x a dbo:Bird . 
    ?x rdfs:label ?label .
    OPTIONAL{?x dbp:status ?status .}
    FILTER ( lang(?label) = "en" )
    
} 

    """
               ) 

sparql_2 = SPARQLWrapper2("http://dbpedia.org/sparql")
sparql_2.setQuery("""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT * WHERE {
    ?x a dbo:Bird . 
    ?x rdfs:label ?label .
    OPTIONAL{?x dbp:status ?status .}
    FILTER ( lang(?label) = "en" )
    
} Limit 5000 Offset 10000

    """
               ) 

In [13]:
inicio = time.time()

dicc_birds_EN_1 = []
url_birds_1 = []
status_birds_1 = []
for result in sparql_1.query().bindings:
    dicc_birds_EN_1.append(f"{result['label'].value}")
    url_birds_1.append(f"{result['x'].value}")
    if len(result) == 3:
        status_birds_1.append(f"{result['status'].value}")
    else: 
        status_birds_1.append('NaN')
    
dicc_birds_EN_2 = []
url_birds_2 = [] 
status_birds_2 = []
for result in sparql_2.query().bindings:
    dicc_birds_EN_2.append(f"{result['label'].value}")
    url_birds_2.append(f"{result['x'].value}")
    if len(result) == 3:
        status_birds_2.append(f"{result['status'].value}")
    else: 
        status_birds_2.append('NaN')

fin = time.time()
print('Tiempo de ejecución del código: ', round(fin-inicio,2), 'seg')

Tiempo de ejecución del código:  2.46 seg


In [14]:
dicc_birds_EN = dicc_birds_EN_1+dicc_birds_EN_2
url_birds = url_birds_1+url_birds_2
status_birds = status_birds_1+status_birds_2

In [15]:
# Finalmente, en la segunda consulta solo se han recopilado 888 pájaros. 

print(len(dicc_birds_EN))
print('---------------')
print(len(url_birds))
print('---------------')
print(len(status_birds))

10898
---------------
10898
---------------
10898


In [16]:
from owlready2 import *

#Creamos una ontología
onto = get_ontology("http://test.org/onto.owl")

In [17]:
with onto: 
    class Blog_1(Thing):
        pass
    class Bird_1(Thing):
        pass
    class mentions_the(ObjectProperty):
        domain = [Blog_1]
        range = [Bird_1]
        
with onto: 
    class mentions_the(Blog_1 >> Bird_1):
        pass
    
with onto: 
    class Blog_1(Thing):
        pass
    class Bird_1(Thing):
        pass
    class is_mentioned_in(ObjectProperty):
        domain = [Bird_1]
        range = [Blog_1]
#         inverse_property = mentions_the 
    
with onto: 
    class is_mentioned_in(Bird_1 >> Blog_1):
        pass

with onto:
    class bird_status(DataProperty, FunctionalProperty):
        domain = [Bird_1]
        range = [str]

# INTENTE DEFINIR UNA PROPIEDAD INVERSA ENTRE EL BLOG MENCIONA 'X' PÁJAROS Y LOS PÁJAROS SALEN MENCIONADOS EN 'X' BLOGS
# PERO FINALMENTE LO HE DEFINIDO COMO DOS "OBJECTPROPERTY" INDEPENDIENTES. 


In [18]:
inicio = time.time()

path = '../Pràctica_2/data/'
data_docs = os.listdir(path)

for doc in data_docs:
    new_path = os.path.join(path,doc)
    txt = open(new_path, 'r')
    txt = txt.readlines()
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub(r'[^\w\s]',' ', txt) # Elimina signes de punutació
    txt = re.sub(r' n ','', txt) # Elimina ' n ', que anteriorment eren \n = 'intro' en el text
    for bird in dicc_birds_EN:
        bird_1 = bird.lower()
        bird_1= re.sub(r'[^\w\s]',' ', bird_1)
        bird_name_ontology = re.sub(r' ', '_', bird)
        bird_1 = ' '+bird_1+'(s|es)? ' # Es defineix una expresió regular per cada ocell
        exp_reg = re.compile(bird_1)
        find_bird = exp_reg.findall(txt)
        if len(find_bird) > 0:
            pos_en_dicc = dicc_birds_EN.index(bird) # Consultam la posició d'aquest ocell dintre de la llista amb tots els ocells
            url = url_birds[pos_en_dicc] # Per despres trobar la seva URL
            status = status_birds[pos_en_dicc]
            
            # Ontology
            text_ontology = Blog_1(doc)
            bird_ontology = Bird_1(bird_name_ontology)
            text_ontology.mentions_the.append(bird_ontology)
            bird_ontology.comment = url # Guardamos como un comentario la url
      
            text_ontology_2 = Blog_1(doc)
            bird_ontology_2 = Bird_1(bird_name_ontology)
            bird_ontology_2.is_mentioned_in.append(text_ontology_2)
            if status != 'NaN':
                bird_ontology.bird_status = status
    
            
            print('En el documento: ', doc,', se menciona', bird)
            print('### El', bird, 'se encuentra en estado', status)
            print()
            print('Para más información consulta en:')
            print(url)
            print('-'*40)

#         if bird in txt: 
#             print('En el documento: ', doc,', se menciona', bird)
print('-'*40)
fin = time.time()
print('Tiempo de ejecución del código: ', round(fin-inicio,2), 'seg')

En el documento:  B_Txt_bird_2.txt , se menciona Belted kingfisher
### El Belted kingfisher se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Belted_kingfisher
----------------------------------------
En el documento:  B_Txt_bird_2.txt , se menciona Black-crowned night heron
### El Black-crowned night heron se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Black-crowned_night_heron
----------------------------------------
En el documento:  B_Txt_bird_2.txt , se menciona Green-winged teal
### El Green-winged teal se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Green-winged_teal
----------------------------------------
En el documento:  B_Txt_bird_2.txt , se menciona Northern gannet
### El Northern gannet se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Northern_gannet
----------------------------------------
En el documento:  B_Tx

En el documento:  E_Txt_bird_5.txt , se menciona Peach-fronted parakeet
### El Peach-fronted parakeet se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Peach-fronted_parakeet
----------------------------------------
En el documento:  E_Txt_bird_5.txt , se menciona White-throated kingbird
### El White-throated kingbird se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/White-throated_kingbird
----------------------------------------
En el documento:  E_Txt_bird_5.txt , se menciona Hyacinth macaw
### El Hyacinth macaw se encuentra en estado VU

Para más información consulta en:
http://dbpedia.org/resource/Hyacinth_macaw
----------------------------------------
En el documento:  E_Txt_bird_5.txt , se menciona Parrot
### El Parrot se encuentra en estado NaN

Para más información consulta en:
http://dbpedia.org/resource/Parrot
----------------------------------------
En el documento:  F_Txt_bird_6.txt , se menciona

En el documento:  G_Txt_bird_7.txt , se menciona Northern wheatear
### El Northern wheatear se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Northern_wheatear
----------------------------------------
En el documento:  G_Txt_bird_7.txt , se menciona Boreal chickadee
### El Boreal chickadee se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Boreal_chickadee
----------------------------------------
En el documento:  G_Txt_bird_7.txt , se menciona Lark bunting
### El Lark bunting se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Lark_bunting
----------------------------------------
En el documento:  G_Txt_bird_7.txt , se menciona Lazuli bunting
### El Lazuli bunting se encuentra en estado LC

Para más información consulta en:
http://dbpedia.org/resource/Lazuli_bunting
----------------------------------------
En el documento:  G_Txt_bird_7.txt , se menciona Ruff
### El Ruff s

In [19]:
# Guardam la ontologia 

onto.save(file=r"C:\Users\Usuario\Documents\MADM\2n Semestre\Mineria de Dades i Text\ontology_birds.xml", format = "rdfxml")

!cat \Users\Usuario\Documents\MADM\2n Semestre\Mineria de Dades i Text\ontology_birds.xml

cat: '\Users\Usuario\Documents\MADM\2n': No such file or directory
cat: 'Semestre\Mineria': No such file or directory
cat: de: No such file or directory
cat: Dades: No such file or directory
cat: i: No such file or directory
cat: 'Text\ontology_birds.xml': No such file or directory
